The HW2VEC codebase is available at https://github.com/AICPS/hw2vec

Below are the steps that worked after many iterations of trials and errors.

**Step 1:** To set up the environment for HW2VEC, Anaconda was installed in the system and the following bash commands were executed in sequence in the terminal:

`conda create --name hw2vec_python37_pyg_pygraphviz`

`conda activate hw2vec_python37_pyg_pygraphviz`

`conda install python=3.7`

`conda install pyg -c pyg`

`python3 -m pip install pygraphviz`

`python3 -m pip install pydot pathlib patool pyunpack pyverilog`

`python3 -m pip install pytorch-nlp`

`python3 -m pip install scikit-image`

`python3 -m pip install pandas networkx matplotlib`

`conda list`

*Note:* 
1. The package versions and the commands mentioned in the HW2VEC repo cause lot of errors as they are quite old. 

2. My system has CUDA 12.

**Step 2:** Modify line 105 of hw2vec/graph2vec/models.py as:

`self.load_state_dict(torch.load(model_weight_path, map_location=torch.device('cpu')),strict=False)`

**Step 3:** Modify line 28 of hw2vec/examples/use_case_1.py as:

`hw_design_dir_path = Path("../assets/datasets/TJ-RTL-toy/TjFree/RS232/")`

Here, `RS232` is the directory containing the circuit-implementation of **RS232**. None of the circuits inside this directory can have the title `topModule.v`. Else, the `pre-process` function of `HW2GRAPH` class in `.hw2vec/hw2graph.py` can't execute on them.

**Step 4:** Unzip the assets/datasets.zip

**Step 5:** Execute the following command in the terminal:

`python use_case_1.py`

## Pyverilog

I setup the environment for installing Pyverilog in my system. The reason for doing this was to see if Pyverilog can process System Verilog files.

For installation steps, I referred to the [Pyverilog repo](https://github.com/PyHDI/Pyverilog) in Github.

I took an example file - *test.v* (available in the above repo) - and changed its extension to .sv. Then, I ran the `VerilogDataflowAnalyzer()` (provided by Pyverilog) on this file by running the following command available in the Pyverilog Github repo:

`python3 pyverilog/examples/example_dataflow_analyzer.py -t top test.sv`

The included files can have either ***.sv*** or ***.v*** extension as long as the extension is mentioned in the `include` statement. 

I ran the following command to run the `VerilogDataflowAnalyzer()` on the *.sv* files of Ariane's `axi_to_apb` module. 

`python3 ./examples/example_dataflow_analyzer.py ../hw2vec/assets/datasets/TJ-RTL-toy/TjIn/ariane_axi_to_apb/cva6_axi2apb_64_32_T3P4.sv ../hw2vec/assets/datasets/TJ-RTL-toy/TjIn/ariane_axi_to_apb/fifo_v3.sv ../hw2vec/assets/datasets/TJ-RTL-toy/TjIn/ariane_axi_to_apb/fifo_v2.sv ../hw2vec/assets/datasets/TJ-RTL-toy/TjIn/ariane_axi_to_apb/fifo_v1.sv ../hw2vec/assets/datasets/TJ-RTL-toy/TjIn/ariane_axi_to_apb/axi_w_buffer.sv ../hw2vec/assets/datasets/TJ-RTL-toy/TjIn/ariane_axi_to_apb/axi_single_slice.sv ../hw2vec/assets/datasets/TJ-RTL-toy/TjIn/ariane_axi_to_apb/axi_r_buffer.sv ../hw2vec/assets/datasets/TJ-RTL-toy/TjIn/ariane_axi_to_apb/axi_b_buffer.sv ../hw2vec/assets/datasets/TJ-RTL-toy/TjIn/ariane_axi_to_apb/axi_aw_buffer.sv ../hw2vec/assets/datasets/TJ-RTL-toy/TjIn/ariane_axi_to_apb/axi_ar_buffer.sv`

But, the Pyverilog parser seems to fail at a number of lines in the *.sv* files. It fails at many special constructs of the language. The following statements are some example lines from the Ariane code at which Pyverilog shows error.

`input  logic [AXI_NUMBYTES-1:0][7:0]   WDATA_i    ,`

`logic [1:0][31:0]              WDATA; `

 `enum logic [3:0] { IDLE, SINGLE_RD, SINGLE_RD_64, BURST_RD_1, BURST_RD, BURST_RD_64, BURST_WR, BURST_WR_64,
                    SINGLE_WR,SINGLE_WR_64, WAIT_R_PREADY, WAIT_W_PREADY} CS, NS;`
                    
 The errors go away when these lines were commented off from the code. But, these lines are part of the actual Ariane implementation.

**Just to be sure, I also ran the *.sv* files of Ariane's `axi_to_apb` module through the HW2VEC tooflow. It fails showing the same error as above.**

**However, both HW2VEC and Pyverilog successfully parsed through simpler *.v* and *.sv* codes without any error.** 

Like this hardware trojan detection method, there will be many other detection methods which not only lack in being tested for large-scale SoC-level designs but also in their ability to compile designs which use advanced features of the HDL.